In this notebook, we apply our final CNN model to the test set. 

In [11]:
## For data handling
import pandas as pd
import numpy as np
import os, shutil

## For plotting
import matplotlib.pyplot as plt
from seaborn import set_style

from sklearn.metrics import accuracy_score, confusion_matrix

## This sets the plot style
## to have a grid on a white background
set_style("darkgrid")

In [2]:
## Importing the things we need from keras
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
from keras.utils.np_utils import to_categorical

from keras.preprocessing.image import ImageDataGenerator
#from keras.utils import 
# tf.keras.utils.image_dataset_from_directory

# New: this will save the training history to a csv file:
from keras.callbacks import CSVLogger

In [3]:
# Set the shape of the images to be loaded into the model:

pixel_height = 219     # This is the FIRST coordinate in the files
pixel_width = 269      # This is the SECOND coordinate in the files
channels = 3           # 3 because we're using RGB


# Type in the paths to the train, validation, and test directories here:

# train_dir = 'spectrograms_cnn_base/large_train' # we no longer use a validation set
test_dir = '../Data/spectrograms_cnn_base/test'
#validation_dir = 'spectrograms_cnn_base/validation'

In [5]:
# Import the final model:

model = models.load_model('Final_model_trained/pathname_model_final')

2022-06-04 10:59:01.371121: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Create a generator that will import and preprocess the spectrograms in the test folder: 

test_datagen = ImageDataGenerator(rescale = 1./255)  # rescale so values are between 0 and 1


batch_size = 1200 #Same as the number of images in the test set. 




test_generator = test_datagen.flow_from_directory(
                    test_dir,
                    target_size = (pixel_height, pixel_width),
                    batch_size = batch_size,
                    class_mode = 'categorical')



Found 1200 images belonging to 6 classes.


In [7]:
# Get the output of test_generator:

output = test_generator.next()

In [8]:
# Divide output into X_test, y_test:

X_test = output[0]
y_test = output[1]

In [9]:
# Get the predicted values on X_test, turn them to integers from 0 to 5:

pred = np.argmax(model.predict(X_test), axis=1)

38/38 [==============================] - 13s 322ms/step


In [13]:
# Turn the one-hot encodings in y_test into integers from 0 to 5:

y_test_cats = np.argmax(y_test, axis=1)

In [12]:
# Get the mapping between integers to emotion labels:

test_generator.class_indices


{'ANG': 0, 'DIS': 1, 'FEA': 2, 'HAP': 3, 'NEU': 4, 'SAD': 5}

In [23]:
# Get dict with int to emotion label:

dict = {0:'ANG', 1:'DIS', 2:'FEA', 3:'HAP', 4:'NEU', 5:'SAD'}

In [27]:
# Get copies of pred, y_test_cats with the emotional labels:

pred_list = list(pred)

y_test_cats_list = list(y_test_cats)


pred_list_emos = [dict[num] for num in pred_list]


y_test_cats_emos = [dict[num] for num in y_test_cats_list]



In [29]:
# Get the confusion matrix, with labels in the same order as the confusion matrix for SVM:
labels = ['NEU', 'ANG', 'HAP', 'SAD', 'FEA', 'DIS']

conf_mat = confusion_matrix(y_test_cats_emos, pred_list_emos, labels=labels )

print(conf_mat)

[[ 87   1  40  74   4   9]
 [ 15 133  45   1   0   3]
 [ 29  41 101  13   8   5]
 [ 18   3  19 135   8  14]
 [ 19  11  77  62  19   9]
 [ 23  28  38  67   2  39]]


In [30]:
# Get the overall accuracy:
acc = accuracy_score(y_test_cats_emos, pred_list_emos)
print(acc)

0.42833333333333334


In [ ]:
# Redraw the confusion matrix to have emotion labels on the axes: